# Convolutions

This notebook walks through part of the subsection "Convolution Continuous and Discrete Random variables" in the [random_variables0.ipynb](https://github.com/ligonteaching/ARE212_Materials/blob/master/random_variables0.ipynb) notebook. The goal of this section is to create a new random variable that is the sum of a continuous random variable $X$ and a discrete random variable $S$ so that:

$Y = X+S$

To do this, we will create a new class that _inherits_ from an already existing class: ``iid.rv_continuous``. So, what is inheritance?

## Inheritance

Inheritance is when we define a class that inherits all the methods and properties from another class. The class being inherited from (in this case, ``iid.rv_continuous``), is called the _parent_ or _base_ class. The class that inherits is the _child_ or _derived_ class.

For example, suppose we had a class called ``Student``, that like in our past class, has attributes like ``Student.grades`` and ``Student.studentID``, and methods such as ``Student.calculateGPA()``. But we also might want to be more specific about what kind of student we are talking about. So we could create a child class called ``GraduateStudent`` that inherits all these attributes and methods from ``Student``, but also adds some like ``GraduateStudent.dissertationChair`` or ``GraduateStudent.fields``.

This is the beauty of inheritance; instead of programming grades attributes for all sorts of student types, we just inherit this structure for all subtypes. 

**This is what Ethan is doing in his convolution class: he knows the convolution will be continuous, so he inherits the structure from the ``iid.rv_continuous`` class, and makes some modifications to make the class convolution-specific.**

### Implementing inheritance

**Step 1: Tell python who you want to inherit from**
Telling python "who" (what class) you are inheriting from is straightforward: just make the class definition a function of the parent class! Ethan does this in this line:

``class ConvolvedContinuousAndDiscrete(iid.rv_continuous):``

**Step 2: Inheriting methods, attributes frm the parent**

Our work isn't quite over though. Remember: the constructor makes the class. So we want to put a method in the constructor that instructs it to bring in all the methods and attributes from the parent class. This is done with the line:

``super(ConvolvedContinuousAndDiscrete, self).__init__(name="ConvolvedContinuousAndDiscrete")``

Don't worry too much about the name part; he's just renaming the class and this could have been omitted. Let's play around with different iterations on this code to get a feel for what's going on:

In [12]:
from scipy.stats import distributions as iid
import numpy as np

Omega = (-1,0,1)
Pr = (1/3.,1/2.,1/6.)

s = iid.rv_discrete(values=(Omega,Pr))
x = iid.norm()

In [7]:
class ConvolvedContinuousAndDiscrete(iid.rv_continuous):

    """Convolve (add) a continuous rv x and a discrete rv s,
       returning the resulting cdf."""
    # note that the constructor is a fn of 2 inputs: 
    # we need to feed in Z and S, the RVs that will be added!
    def __init__(self,f,s):  
        self.continuous_rv = f
        self.discrete_rv = s
        super(ConvolvedContinuousAndDiscrete, self).__init__() # note: omitting name change
        
y = ConvolvedContinuousAndDiscrete(x, s)
print(y.continuous_rv)
print(y.discrete_rv)
print(y.a)

-inf


Whoa! Where did the attribute ``y.a`` come from?? It was inherited from ``iid.rv_continuous`` thanks to the ``super()`` constructor! Note that we need ``super()`` for this to work:

In [8]:
# Modification 1: No super() method
class ConvolvedContinuousAndDiscrete(iid.rv_continuous):

    """Convolve (add) a continuous rv x and a discrete rv s,
       returning the resulting cdf."""
    # note that the constructor is a fn of 2 inputs: 
    # we need to feed in Z and S, the RVs that will be added!
    def __init__(self,f,s):  
        self.continuous_rv = f
        self.discrete_rv = s
        
y = ConvolvedContinuousAndDiscrete(x, s)
print(y.continuous_rv)
print(y.discrete_rv)
print(y.a)

AttributeError: 'ConvolvedContinuousAndDiscrete' object has no attribute 'a'

### Overwriting generic functions

The ``iid.rv_continuous`` class has generic functions ``_pdf`` and ``_cdf`` that are meant to be overwritten by the child class. The underscore here is just a naming convention; it signals that the method is "private". This is not important. Here we want to overwrite the CDF. We know that:

$$F(y) = \sum_{s} F_X(y - s)P_S(s)$$

That is, we are summing up the probabilities of $S = s$, times the probability that $X \leq y - s$ ($\Rightarrow X + s \leq y$). So, the ingredients we need are:

1. $F_X()$ The CDF of X evaluated at y. Since X is an instance of ``iid.continuous``, we know it has a ``cdf()`` method.
2. $\{P_S(s)\} s = 1,\ldots, K$ The probabilities of each mass point in the support of S. Since S is  we know if has an attribute ``pk``. 
3. $\{s\} s = 1,\ldots, K$ The values S takes on. Again, as an instance of ``iid.discrete``, S has the attribute ``xk``, the different values for which S has positive probability.

With these elements, we will loop over the elements of $S$, adding each new calculation to the one before it:

In [11]:
# Modification 1: No super() method
class ConvolvedContinuousAndDiscrete(iid.rv_continuous):

    """Convolve (add) a continuous rv x and a discrete rv s,
       returning the resulting cdf."""
    # note that the constructor is a fn of 2 inputs: 
    # we need to feed in Z and S, the RVs that will be added!
    def __init__(self,f,s):  
        self.continuous_rv = f
        self.discrete_rv = s
        super(ConvolvedContinuousAndDiscrete, self).__init__() # note: omitting name change
        
    def _cdf(self,z):
        F=0
        s = self.discrete_rv
        x = self.continuous_rv
        # loop over values of S
        for k in range(len(s.xk)):
            F = F + x.cdf(z-s.xk[k])*s.pk[k] 
            # each time, add CDF to former value (started at 0 above)
            # F_X(y-s)P_S(s)
        return F
    
y = ConvolvedContinuousAndDiscrete(x, s)
print(y.cdf(2))

0.9617324256934575


In [19]:
# Modification 2: an alternative implementation of the CDF method
class ConvolvedContinuousAndDiscrete(iid.rv_continuous):

    """Convolve (add) a continuous rv x and a discrete rv s,
       returning the resulting cdf."""
    # note that the constructor is a fn of 2 inputs: 
    # we need to feed in Z and S, the RVs that will be added!
    def __init__(self,f,s):  
        self.continuous_rv = f
        self.discrete_rv = s
        super(ConvolvedContinuousAndDiscrete, self).__init__() # note: omitting name change
        
    def _cdf(self,z):
        F=0
        s = self.discrete_rv
        x = self.continuous_rv
        # loop over values of S
        for k in range(len(s.xk)):
            F = F + x.cdf(z-s.xk[k])*s.pk[k] 
            # each time, add CDF to former value (started at 0 above)
            # F_X(y-s)P_S(s)
        return F     
    
    def cdf1(self,z):
        s = self.discrete_rv
        x = self.continuous_rv
        # loop over values of S
        return np.sum([x.cdf(z-s.xk[k])*s.pk[k] for k in range(len(s.xk))])
    
    def cdf2(self,z):
        F=0
        s = self.discrete_rv
        x = self.continuous_rv
        
        for sval, p in zip(s.xk, s.pk):
            F = F + x.cdf(z - sval)*p
        return F

y = ConvolvedContinuousAndDiscrete(x, s)
print(y.cdf(2))
print(y.cdf1(2))
print(y.cdf2(2))

0.9617324256934575
0.9617324256934575
0.9617324256934575
